In [1]:
from typing import Annotated, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, trim_messages,filter_messages
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.checkpoint.memory import MemorySaver

# 기본 ollama_langchain 코드

In [2]:
# 기본 ollama_langchain 코드
template = """Question: {question}
Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="qwen2.5")
chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

'LangChain appears to be a term that could refer to several different concepts depending on the context, but it doesn\'t seem to be an established or widely recognized term in the field of artificial intelligence (AI) or related technologies as of my last update in October 2023. Let\'s break this down step by step:\n\n1. **Contextual Clues**:\n   - "Lang" could potentially refer to language or a specific programming language, such as Python.\n   - "Chain" might indicate a concept involving chaining or connecting multiple components.\n\n2. **Possible Interpretations**:\n   - **Language Chains**: This could be related to creating sequences of natural language tasks or processes.\n   - **AI/ML Context**: In the context of AI and machine learning, it might refer to a method or framework for chaining together different models or layers, similar to how blocks in a lego set are connected.\n   - **Coding Frameworks**: It could be a hypothetical name for a coding framework that involves chains 

---

### 메모리 저장코드

In [3]:
# 메모리 저장 코드
class State(TypedDict):
    messages: Annotated[list, add_messages]
    
    
builder = StateGraph(State)

model = OllamaLLM(model="qwen2.5")

def chatbot(state: State):
    answer = model.invoke(state['messages'])
    return {'messages':[answer]}

### 챗봇 노드 추가
- 첫 번째 인자는 고유한 노드 이름
- 두 번째 인자는 실행할 함수 또는 runnable 객체

In [4]:
# 챗봇 노드 추가
# 첫 번째 인자는 고유한 노드 이름
# 두 번쨰 인자는 실행할 함수 또는 runnable 객체
builder.add_node("chatbot", chatbot)

#엣지 추가
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

### 그래프 컴파일 및 사진 추출

In [5]:
# 그래크 컴파일
graph = builder.compile()

# 그래프 사진 추출
graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00j\x00\x00\x00\xea\x08\x02\x00\x00\x00\xc5\xf3G\x18\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x16\xdeIDATx\x9c\xed\x9di`\x14E\xda\xc7k\xba\xe7>3\x99\x90c&\t\x99\\\x04\x92\x001`\xb2q\x97p\x04QN\x159\xc4\x97\x95\x85w\x05Y\x0e\x05\x17aQ\x16\xaf\xd5\x85E@\r\x88 \x04a\x05E1\x08\x08$"\xd9\xe5\\\x08\xd1\x84@ LNrg\x8ed\xee\xa3\x8f\xf7C\xfb\x86\xac\xce\x99\x9e!=\xb1\x7f\x9f&S\xd5=O\xff\xd3]U\xfd\xd4S\xf50p\x1c\x074}\x05\xeao\x03\x82\x1bZ>R\xd0\xf2\x91\x82\x96\x8f\x14\xb4|\xa4\xa0\xe5#\x05\x93\xe4\xf1\x06\xad\xa3[\xe30\x1bP\xb3\x1eE\x1c8\x86\x05\xc10\x88\xcd\x858<\x88/\x82\x05\x12f\x98\x9cC\xe6T\x8c\xbe\x8d\xfb4\xad\xb6\x9a\nS\xdd\r\x13\x9b\xcf\x008\x83/\x82\xf9b\x98\'`bh\x10\xc8\x07\xc1\xa0\xab\xd3a6\xa0\\>\xd4RkU\xa6\t\x12\xd2\x05\xd1\xc9\xfc>\x9c\xcag\xf9\x8c]\xc8\xa5\xe3j\x1c\x80\x900\x962]\x10\x1e\xcd\xed\xc3\xafR\x07\x83\xceQWi\xeah\xb2u\xb5;~3M\xa6H\xe0\xf9t\xb8o\xf2]+\xd2V^\xea\xce\x99\x166$S\xe4\xbb\xa9\x94\xa6\xb5\xder\xf9\xb8F\x1a\

### 스트림 메소드 실행 (실행시키면 메모리 저장 안됨)

In [6]:
# 스트림 메소드 실행 (실행시키면 메모리 저장 안됨)
input = {'messages':[HumanMessage('안녕하세요')]}
for chunk in graph.stream(input):
    print(chunk)

{'chatbot': {'messages': ['안녕하세요! 어떻게 도와드릴 수 있을까요?']}}


# !체크포인터 추가

In [7]:
# 체크포인터 추가
graph = builder.compile(checkpointer=MemorySaver())

In [8]:
# 스트림 메소드 재실행하여 메모리 기능 확인
thread1 = {'configurable':{'thread_id':1}}

# 영속성 추가 후 그래프 실행
result_1 = graph.invoke({
    'messages':[HumanMessage('안녕하세요, 저는 민혁입니다!')]
}, thread1)
result_2 = graph.invoke({
    'messages':[HumanMessage('제 이름은 뭐죠?')]
},thread1)

In [9]:
# 상태 확인
graph.get_state(thread1) 

StateSnapshot(values={'messages': [HumanMessage(content='안녕하세요, 저는 민혁입니다!', additional_kwargs={}, response_metadata={}, id='26f0bbb1-ea08-4896-9441-75536fa44184'), HumanMessage(content='안녕하세요, 민혁님! 만나서 반갑습니다. 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}, id='2b104b7d-819e-4d7c-9c83-ef4d44eb64b9'), HumanMessage(content='제 이름은 뭐죠?', additional_kwargs={}, response_metadata={}, id='91364f58-c247-47a3-bcf0-1934f5cc9e6e'), HumanMessage(content='안녕하세요, 민혁님! 당신의 이름은 이미 말씀하셨으니 바로 민혁님이라는 거예요. 궁금한 사항이 더 있으신가요? 도와드릴 수 있는 것이 있으면 언제든지 알려주세요!', additional_kwargs={}, response_metadata={}, id='60a12043-6089-496a-83b0-797c1b28d7a5')]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f034740-bf00-6b84-8004-2ab3a117f5a4'}}, metadata={'source': 'loop', 'writes': {'chatbot': {'messages': ['안녕하세요, 민혁님! 당신의 이름은 이미 말씀하셨으니 바로 민혁님이라는 거예요. 궁금한 사항이 더 있으신가요? 도와드릴 수 있는 것이 있으면 언제든지 알려주세요!']}}, 'step': 4, 'parents': {}, 'thread_id': 1}, created_at='2025-05-19T05:

---
# 샘플 메시지 몇개 작성
- 필터 적용

In [ ]:
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다'),
    HumanMessage(content='안녕하세요! 나는 민혁입니다.'),
    AIMessage(content='안녕하세요'),
    HumanMessage(content='바닐라 아이스크림을 좋아해요?'),
    AIMessage(content='좋네요!'),
    HumanMessage(content='2+2는 얼마죠?'),
    AIMessage(content='4입니다'),
    HumanMessage(content='고마워요'),
    AIMessage(content='천만에요!'),
    HumanMessage(content='즐거운가요?'),
    AIMessage(content='예!'),
]


trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=OllamaLLM(model="qwen2.5"),
    include_system=True,
    allow_partial=False,
    start_on='human'
)

trimmed = trimmer.invoke(messages)
print(trimmed)

c:\Users\ljh10\anaconda3\envs\ollama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ljh10\anaconda3\envs\ollama_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ljh10\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mo

[SystemMessage(content='당신은 친절한 어시스턴트입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='즐거운가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='예!', additional_kwargs={}, response_metadata={})]


### 사용자 메시지 필터링
- include_type : 메세지 유형
- exclude_names : 해당 이름 제외

In [17]:
message = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다',id=1),
    HumanMessage(content='예시 입력',id=2, name='example_user'),
    AIMessage(content='예시 출력',id=3, name='example_assistant'),
    HumanMessage(content='실제 입력', id=4, name='bob'),
    AIMessage(content='실제 출력', id=5, name='alice')
]

# 사용자 메시지만 필터링
human_messages = filter_messages(messages, include_types='human')
human_messages

[HumanMessage(content='안녕하세요! 나는 민혁입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='바닐라 아이스크림을 좋아해요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2+2는 얼마죠?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='고마워요', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='즐거운가요?', additional_kwargs={}, response_metadata={})]